In [39]:
import pandas as pd
import numpy as np
import csv
import random

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.base import BaseEstimator
from scipy.optimize import minimize
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier
from sknn.mlp import Classifier, Layer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

DATA_DIR = "/Users/patrickkennedy/Desktop/Data_Science/Numerai/"
train = pd.read_csv(DATA_DIR + "numerai_training_data.csv")
test = pd.read_csv(DATA_DIR + "numerai_tournament_data.csv")

#upload file with two columns ala kaggle: t_id, probability


Couldn't import dot_parser, loading of dot files will not be possible.


In [40]:
target = train['target'].values

t_id = test['t_id']

In [41]:
holder = []
preds = []

In [35]:
#Defining the classifiers
#think about jittering the random state and then averaging the predictions together ...
#only good for extra trees, RF?, native XGB, NN
clfs = {#'LR'  : LogisticRegression(), 
        #'SVM' : SVC(probability=True, random_state=random_state), 
        #'RF'  : RandomForestClassifier(n_estimators=100, n_jobs=-1), 
        #'GBM' : GradientBoostingClassifier(n_estimators=50), 
        'ETC' : ExtraTreesClassifier(n_estimators=100, n_jobs=-1),
        #'KNN' : KNeighborsClassifier(n_neighbors=5),
        'XGBc': XGBClassifier(objective='binary:logistic') #,
        #'NN'  : Pipeline([('min/max scaler', MinMaxScaler(feature_range=(-1.0, 1.0))),
        #                  ('neural network', Classifier(layers=[Layer("Rectifier", units=10),
        #                                                        Layer("Tanh", units=10),
        #                                                        Layer("Softmax")], 
        #                                                n_iter=5))])       
       }


for_real = True

#predictions on the validation and test sets
p_valid = []
p_test = []

p_valid_real = []
p_test_real = []
   
print('Performance of individual classifiers (1st layer) on X_test')   
print('------------------------------------------------------------')

#Spliting data into train and test sets.
X, X_test, y, y_test = train_test_split(train.drop(['target'], axis=1), target, test_size=0.2)
    
#Spliting train data into training and validation sets.
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25)

X_real = train.drop(['target'], axis=1)
X_test_real = test.drop(['t_id'], axis=1)
y_real = target
    
X_train_real, X_valid_real, y_train_real, y_valid_real = train_test_split(X_real, y_real, test_size=0.25)

random.seed(117)

for nm, clf in clfs.items():
    if nm == 'NN':
        #First run. Training on (X_train, y_train) and predicting on X_valid.
        clf.fit(X_train.as_matrix(), y_train)
        yv = clf.predict_proba(X_valid.as_matrix())
        p_valid.append(yv)
        
        #Second run. Training on (X, y) and predicting on X_test.
        clf.fit(X.as_matrix(), y)
        yt = clf.predict_proba(X_test.as_matrix())
        p_test.append(yt)
        
        if for_real:
            #First run. Training on (X_train, y_train) and predicting on X_valid.
            clf.fit(X_train_real.as_matrix(), y_train_real)
            yv_real = clf.predict_proba(X_valid_real.as_matrix())
            p_valid_real.append(yv_real)
        
            #Second run. Training on (X, y) and predicting on X_test.
            clf.fit(X_real.as_matrix(), y_real)
            yt_real = clf.predict_proba(X_test_real.as_matrix())
            p_test_real.append(yt_real)
    
    else:
        #First run. Training on (X_train, y_train) and predicting on X_valid.
        clf.fit(X_train, y_train)
        yv = clf.predict_proba(X_valid)
        p_valid.append(yv)
        
        #Second run. Training on (X, y) and predicting on X_test.
        clf.fit(X, y)
        yt = clf.predict_proba(X_test)
        p_test.append(yt)
        
        if for_real:
            #First run. Training on (X_train, y_train) and predicting on X_valid.
            clf.fit(X_train_real, y_train_real)
            yv_real = clf.predict_proba(X_valid_real)
            p_valid_real.append(yv_real)
        
            #Second run. Training on (X, y) and predicting on X_test.
            clf.fit(X_real, y_real)
            yt_real = clf.predict_proba(X_test_real)
            p_test_real.append(yt_real)
       
    #Printing out the performance of the classifier
    print('{:10s} {:2s} {:1.7f}'.format('%s: ' %(nm), 'logloss  =>', log_loss(y_test, yt)))
print('')


#when running the full data
#take out the logloss function... or alternatively, run both the split data and full data model so that
#i can compare my training logloss vs kaggle logloss


#maybe build a system that tries base models, shakes random states in order to find uncorrelated models?
#don't know why but my bayes_opt script is giving a worse score than untuned model (default params)

Performance of individual classifiers (1st layer) on X_test
------------------------------------------------------------
ETC:       logloss  => 0.7040552
XGBc:      logloss  => 0.6920312



In [44]:
print('Performance of optimization based ensemblers (2nd layer) on X_test')   
print('------------------------------------------------------------')
    
#Creating the data for the 2nd layer.
XV = np.hstack(p_valid)
XT = np.hstack(p_test)  


#XV = p_valid.as_matrix()
#XT = p_test.as_matrix()

n_classes = 2

    
#EN_optA
enA = EN_optA(n_classes)
enA.fit(XV, y_valid)
w_enA = enA.w
y_enA = enA.predict_proba(XT)
print('{:20s} {:2s} {:1.7f}'.format('EN_optA:', 'logloss  =>', log_loss(y_test, y_enA)))
    
#Calibrated version of EN_optA 
cc_optA = CalibratedClassifierCV(enA, method='isotonic')
cc_optA.fit(XV, y_valid)
y_ccA = cc_optA.predict_proba(XT)
print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optA:', 'logloss  =>', log_loss(y_test, y_ccA)))
        
#EN_optB
enB = EN_optB(n_classes) 
enB.fit(XV, y_valid)
w_enB = enB.w
y_enB = enB.predict_proba(XT)
print('{:20s} {:2s} {:1.7f}'.format('EN_optB:', 'logloss  =>', log_loss(y_test, y_enB)))

#Calibrated version of EN_optB
cc_optB = CalibratedClassifierCV(enB, method='isotonic')
cc_optB.fit(XV, y_valid)
y_ccB = cc_optB.predict_proba(XT)
print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optB:', 'logloss  =>', log_loss(y_test, y_ccB)))
print('')


if for_real:
    XV_real = np.hstack(p_valid_real)
    XT_real = np.hstack(p_test_real) 
    
    #EN_optA
    enA_real = EN_optA(n_classes)
    enA_real.fit(XV_real, y_valid_real)
    w_enA_real = enA_real.w
    y_enA_real = enA_real.predict_proba(XT_real)
    #print('{:20s} {:2s} {:1.7f}'.format('EN_optA:', 'logloss  =>', log_loss(y_test, y_enA)))
    
    #Calibrated version of EN_optA 
    cc_optA_real = CalibratedClassifierCV(enA_real, method='isotonic')
    cc_optA_real.fit(XV_real, y_valid_real)
    y_ccA_real = cc_optA_real.predict_proba(XT_real)
    #print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optA:', 'logloss  =>', log_loss(y_test, y_ccA)))
        
    #EN_optB
    enB_real = EN_optB(n_classes) 
    enB_real.fit(XV_real, y_valid_real)
    w_enB_real = enB_real.w
    y_enB_real = enB_real.predict_proba(XT_real)
    #print('{:20s} {:2s} {:1.7f}'.format('EN_optB:', 'logloss  =>', log_loss(y_test, y_enB)))

    #Calibrated version of EN_optB
    cc_optB_real = CalibratedClassifierCV(enB_real, method='isotonic')
    cc_optB_real.fit(XV_real, y_valid_real)
    y_ccB_real = cc_optB_real.predict_proba(XT_real)
    #print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optB:', 'logloss  =>', log_loss(y_test, y_ccB)))
    print('')

Performance of optimization based ensemblers (2nd layer) on X_test
------------------------------------------------------------
EN_optA:             logloss  => 0.6920534
Calibrated_EN_optA:  logloss  => 0.6921747
EN_optB:             logloss  => 0.6920597
Calibrated_EN_optB:  logloss  => 0.6924957




In [46]:
best_score = 10.0

for i in range(10000):
    first = random.randint(0,20)
    second = random.randint(0,20)
    third = random.randint(0,20)
    fourth = random.randint(0,20)
    total = first + second + third + fourth
    first = first / (total * 1.0)
    second = second / (total * 1.0)
    third = third / (total * 1.0)
    fourth = fourth / (total * 1.0)
    
    y_3l = (y_enA * first) + (y_ccA * second) + (y_enB * third) + (y_ccB * fourth)
    current_score = log_loss(y_test, y_3l)
    
    if current_score < best_score:
        print('{:20s} {:2s} {:1.7f}'.format('3rd_layer:', 'logloss  =>', log_loss(y_test, y_3l)))
        #print first, second, third, fourth
        best_score = current_score
        best_first = first
        best_second = second
        best_third = third
        best_fourth = fourth

3rd_layer:           logloss  => 0.6921187
3rd_layer:           logloss  => 0.6921004
3rd_layer:           logloss  => 0.6920914
3rd_layer:           logloss  => 0.6920411
3rd_layer:           logloss  => 0.6920351
3rd_layer:           logloss  => 0.6920240
3rd_layer:           logloss  => 0.6920239
3rd_layer:           logloss  => 0.6920233
3rd_layer:           logloss  => 0.6920232
3rd_layer:           logloss  => 0.6920230
3rd_layer:           logloss  => 0.6920229


In [48]:
preds = (y_enA_real * best_first) + \
        (y_ccA_real * best_second) + \
        (y_enB_real * best_third) + \
        (y_ccB_real * best_fourth)
            
pd.DataFrame({"t_id": t_id, "probability": preds[:,1]}).to_csv('3-level-calibrated-xgbc-extc.csv',index=False)

In [42]:

def objf_ens_optA(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optA ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem (12 in Airbnb competition)
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    w = np.abs(w)
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol += Xs[i] * w[i]
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
        

class EN_optA(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$,  it computes the optimal set of weights
    $w_1, w_2, ..., w_n$; such that minimizes $log\_loss(y_T, y_E)$, 
    where $y_E = X_1*w_1 + X_2*w_2 +...+ X_n*w_n$ and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optA, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #All weights must sum to 1
        cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
        #Calling the solver
        res = minimize(objf_ens_optA, x0, args=(Xs, y, self.n_class), 
                       method='SLSQP', 
                       bounds=bounds,
                       constraints=cons
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be blended.
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The blended prediction.
        """
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred += Xs[i] * self.w[i] 
        return y_pred
    
    

In [43]:
def objf_ens_optB(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optB ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem, i.e. = 12
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    #Constraining the weights for each class to sum up to 1.
    #This constraint can be defined in the scipy.minimize function, but doing 
    #it here gives more flexibility to the scipy.minimize function 
    #(e.g. more solvers are allowed).
    w_range = np.arange(len(w))%n_class 
    for i in range(n_class): 
        w[w_range==i] = w[w_range==i] / np.sum(w[w_range==i])
        
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol[:, i % n_class] += Xs[int(i / n_class)][:, i % n_class] * w[i] 
        
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
    

class EN_optB(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$, where each $X_i$ has
    $m=12$ clases, i.e. $X_i = X_{i1}, X_{i2},...,X_{im}$. The algorithm finds the optimal 
    set of weights $w_{11}, w_{12}, ..., w_{nm}$; such that minimizes 
    $log\_loss(y_T, y_E)$, where $y_E = X_{11}*w_{11} +... + X_{21}*w_{21} + ... 
    + X_{nm}*w_{nm}$ and and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optB, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(self.n_class * len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #Calling the solver (constraints are directly defined in the objective
        #function)
        res = minimize(objf_ens_optB, x0, args=(Xs, y, self.n_class), 
                       method='L-BFGS-B', 
                       bounds=bounds, 
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The ensembled prediction.
        """
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred[:, i % self.n_class] += \
                   Xs[int(i / self.n_class)][:, i % self.n_class] * self.w[i]  
        return y_pred
    
    
